In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import logging
logging.basicConfig(level=logging.INFO)

import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score

import pandas as pd
import pandas

import numpy as np
#import truncator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ON_CLUSTER=False

In [4]:
flags = """
-beta \
-in:file:fullatom \
-multi_cool_annealer 10 \
-renumber_pdb 1 \
-overwrite \
-out:pdb_gz \
-out::file::pdb_comments \
-run:preserve_header \
-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm \
"""

In [5]:
if not ON_CLUSTER:
    pyrosetta.init(flags)
else:
    cluster = truncator.init_env(flags, 10)

INFO:pyrosetta.rosetta:Found rosetta database at: /software/conda/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta devel 2019.01+HEAD.dbc838b6ae620b1293476b1bd4366ffc2facc5b5 2019-01-03T10:31:13] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but -weights are also specified.  Not changing input weights file!
INFO:rosetta:core.init: Rosetta version: 2019.01+HEAD.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
INFO:rosetta:core.init: command: PyRosetta -beta -in:file:fullatom -multi_cool_annealer 10 -renumber_pdb 1 -overwrite -out:pdb_gz -out::file::pdb_comments -run:preserv

In [6]:
def read_file(filename):
    with open(filename, 'r') as myfile:
        data = myfile.read()
    return data        

In [12]:
redesign_protocol = read_file("truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml")
task_design = rosetta_scripts.SingleoutputRosettaScriptsTask(redesign_protocol)
# Syntax check via setup
logging.basicConfig(level=logging.WARN)
task_design.setup()

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...


RuntimeError: 

File: /opt/conda/conda-bld/pyrosetta_1546650963875/work/source/src/protocols/rosetta_scripts/RosettaScriptsParser.cc:1256
Input rosetta scripts XML file "XmlString (not real file)" failed to validate against the rosetta scripts schema. Use the option -parser::output_schema <output filename> to output the schema to a file to see all valid options.
Your XML has failed validation.  The error message below will tell you where in your XML file the error occurred.  Here's how to fix it:

1) If the validation fails on something obvious, like an illegal attribute due to a spelling error (perhaps you used scorefnction instead of scorefunction), then you need to fix your XML file.
2) If you haven’t run the XML rewriter script and this might be pre-2017 Rosetta XML, run the rewriter script (tools/xsd_xrw/rewrite_rosetta_script.py) on your input XML first.  The attribute values not being in quotes (scorefunction=talaris2014 instead of scorefunction="talaris2014") is a good indicato